<a href="https://colab.research.google.com/github/mmnnf/Book_Store_app_with_filter/blob/main/vehicle_model_detection/masin_model_tanima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

print(f"PyTorch  : {torch.__version__}")
print(f"CUDA sür.: {torch.version.cuda}")
print(f"GPU      : {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK'}")

import os, math, json, random, shutil, numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
import torchvision.transforms as T, timm
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import classification_report

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('timm version :', timm.__version__)

# ===============================================================
# 1)  GeM  &  ArcMarginProduct  (fallback)
# ===============================================================
class GeM(nn.Module):
    def __init__(self, p=3., eps=1e-6): super().__init__(); self.p = nn.Parameter(torch.ones(1)*p); self.eps=eps
    def forward(self,x): return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p),
                                            (x.size(-2),x.size(-1))).pow(1./self.p)[:, :, 0, 0]

try:
    from timm.layers import ArcMarginProduct          # timm 0.8+
except ImportError:
    class ArcMarginProduct(nn.Module):                # basit fallback
        def __init__(self,f_in,f_out,s=30.,m=0.4):   super().__init__(); self.s=s; self.m=m
        def forward(self,x,y):                        # sadece logits üretmek yeterli
            return x                                  # eğitim için CrossEntropy kullanıyoruz

# ===============================================================
# 2) Yol & Hyperparam
# ===============================================================
ROOT = 'stanford_cars_extracted'   # drive’daki “train / test”
CKPT = 'car_b4_arc_best.pth'
BATCH, EPOCHS, LR = 4, 60, 3e-4
MEAN,STD = [0.485,0.456,0.406],[0.229,0.224,0.225]
torch.manual_seed(42); np.random.seed(42); random.seed(42)

# ===============================================================
# 3) Dataset
# ===============================================================
class Cars(Dataset):
    def __init__(self, root, split, tf):
        self.root=os.path.join(root, split); self.tf=tf
        self.cls = sorted(os.listdir(self.root))
        self.c2i={c:i for i,c in enumerate(self.cls)}
        self.samples=[(os.path.join(self.root,c,f), self.c2i[c])
                      for c in self.cls
                      for f in os.listdir(os.path.join(self.root,c))
                      if f.lower().endswith(('jpg','png','jpeg'))]
    def __len__(self): return len(self.samples)
    def __getitem__(self,i):
        p,y=self.samples[i]; img=Image.open(p).convert('RGB')
        return self.tf(img), y
    def classes(self): return self.cls

train_tf = T.Compose([
    T.RandomResizedCrop(380,scale=(0.7,1.0),ratio=(0.75,1.33)),
    T.RandomHorizontalFlip(), T.ColorJitter(0.3,0.3,0.3,0.1),
    T.RandomPerspective(0.2), T.ToTensor(), T.Normalize(MEAN,STD)
])
val_tf = T.Compose([T.Resize(400),T.CenterCrop(380),
                    T.ToTensor(),T.Normalize(MEAN,STD)])

full = Cars(ROOT,'train',train_tf)
n_cls = len(full.classes());  print('Sınıf sayısı :', n_cls)
val_len = len(full)//10; train_len=len(full)-val_len
train_ds,val_ds=random_split(full,[train_len,val_len],
                             generator=torch.Generator().manual_seed(42))
val_ds.dataset.tf = val_tf
test_ds = Cars(ROOT,'test', val_tf)

dl_kw=dict(batch_size=BATCH, num_workers=0, pin_memory=False)
train_dl=DataLoader(train_ds,shuffle=True, **dl_kw)
val_dl  =DataLoader(val_ds,shuffle=False,**dl_kw)
test_dl =DataLoader(test_ds,shuffle=False,**dl_kw)

# ===============================================================
# 4) Model  (Eff-B4 + GeM + ArcFace)
# ===============================================================
backbone = timm.create_model('efficientnet_b4',
                             pretrained=True, num_classes=0).to(DEVICE)
backbone.global_pool = GeM()                     # ← GeM havuzu eklendi
head = ArcMarginProduct(backbone.num_features, n_cls,
                        s=30., m=0.4).to(DEVICE)


backbone.global_pool = GeM().to(DEVICE)



criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(
    list(backbone.parameters())+list(head.parameters()),
    lr=LR, weight_decay=1e-2)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type=='cuda'))

# def run(dl, train=True):
#     backbone.train(train); head.train(train)
#     tot=loss_sum=correct=0
#     for x,y in tqdm(dl, leave=False):
#         x,y = x.to(DEVICE), y.to(DEVICE)
#         with torch.cuda.amp.autocast(enabled=(DEVICE.type=='cuda')):
#             feats = backbone(x)
#             logits= head(feats,y)
#             loss  = criterion(logits,y)
#         if train:
#             optimizer.zero_grad(); scaler.scale(loss).backward()
#             scaler.step(optimizer); scaler.update()
#         loss_sum += loss.item()*x.size(0)
#         correct  += (logits.argmax(1)==y).sum().item(); tot+=x.size(0)
#     return loss_sum/tot, correct/tot*100




def run(dl, train=True):
    backbone.train(train)
    head.train(train)

    tot = 0
    loss_sum = 0.0
    correct  = 0

    for step, (x, y) in enumerate(tqdm(dl, leave=False)):
        # ----  DEVICE KONTROLÜ – yalnızca ilk adımda ---------------
        if step == 0:
            print("❯ Batch tensor device :", x.device)
            print("❯ Backbone first weight :", next(backbone.parameters()).device)
            if DEVICE.type == 'cuda':
                print("❯ GPU bellek (MB)     :", round(torch.cuda.memory_allocated() / 1e6, 1))
        # ------------------------------------------------------------

        x, y = x.to(DEVICE), y.to(DEVICE)

        with torch.cuda.amp.autocast(enabled=(DEVICE.type == 'cuda')):
            feats  = backbone(x)
            logits = head(feats, y)
            loss   = criterion(logits, y)

        if train:
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        loss_sum += loss.item() * x.size(0)
        correct  += (logits.argmax(1) == y).sum().item()
        tot      += x.size(0)

    return loss_sum / tot, correct / tot * 100


best=0
for ep in range(1,EPOCHS+1):
    tl,ta = run(train_dl,True)
    vl,va = run(val_dl,False)
    sched.step()
    print(f"E{ep:02d}  tr:{ta:5.1f}%  val:{va:5.1f}%")
    if va>best:
        best=va
        torch.save({'b':backbone.state_dict(),'h':head.state_dict()},
                   CKPT)
        print(f"  ✔ New best {best:.1f}% saved")

# ===============================================================
# 5) Test
# ===============================================================
ckpt=torch.load(CKPT,map_location=DEVICE)
backbone.load_state_dict(ckpt['b']); head.load_state_dict(ckpt['h'])
_,test_acc = run(test_dl,False)
print(f"\nTest top-1 : {test_acc:.2f}%")

y_true=y_pred=[]
backbone.eval(); head.eval()
with torch.no_grad():
    for x,y in test_dl:
        logits=head(backbone(x.to(DEVICE)),y.to(DEVICE))
        y_true.extend(y.numpy()); y_pred.extend(logits.argmax(1).cpu().numpy())
print(classification_report(y_true,y_pred,target_names=full.classes()))
